<a href="https://colab.research.google.com/github/Choe-Useong/Extracting-Structural-Clusters-in-Equity-Markets-via-Correlation-Network-Community-Detection/blob/main/6_EDA_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# EDA of item__로그수익률.parquet

# 1. 데이터 기본 구조 EDA

import pandas as pd

df = pd.read_parquet("item__로그수익률 (1).parquet")

# 관측치 수, 변수 수
n_rows, n_cols = df.shape
print("관측치 수:", n_rows)
print("변수 수:", n_cols)

# 컬럼 목록 확인
print("컬럼 목록:")
print(df.columns.tolist())

df.head()

In [ ]:
# 2. 시간 관련 EDA

df.columns = pd.to_datetime(df.columns)

# 전체 기간 범위
start_date = df.columns.min()
end_date = df.columns.max()

print("전체 기간 범위:")
print("시작일:", start_date)
print("종료일:", end_date)

# 연도 추출
years = df.columns.year

# 연도별 데이터 개수
year_counts = pd.Series(years).value_counts().sort_index()

print("연도별 데이터 개수:")
print(year_counts)

# 연도별 데이터 개수 그래프

import matplotlib.pyplot as plt; plt.show()

year_counts.plot(kind="bar", title="YearCounts")

In [ ]:
# 2015년 제도 변경 전·후 비교

period_2015 = pd.Series(
    ["Prior to 2015" if y < 2015 else "Since 2015" for y in years]
).value_counts()

print("2015년 제도 변경 전·후 컬럼(날짜) 개수:")
print(period_2015)

# 제도 변경 전·후 그래프
period_2015.plot(kind="bar", title="period 2015")

In [ ]:
# 3. 결측치 EDA

# 종목별 결측치 비율
missing_by_item = df.isna().mean(axis=1)

print("종목별 결측치 비율 20개:")
print(missing_by_item.sort_values(ascending=False).head(20))


# 날짜별 결측치 비율
missing_by_date = df.isna().mean(axis=0)

print("\n날짜별 결측치 비율 20개:")
print(missing_by_date.sort_values(ascending=False).head(20))

In [ ]:
# 결측치 비율 0.3 이상인 종목 선택
high_missing_items = missing_by_item[missing_by_item >= 0.3]

# 갯수 출력
print("결측치 비율 ≥ 0.3인 종목 수:", len(high_missing_items))

# 종목 이름과 비율 출력
print("\n결측치 비율 ≥ 0.3인 종목 리스트:")
print(high_missing_items.sort_values(ascending=False))

In [ ]:
# 결측치 비율 0.3 이상인 날짜 선택
high_missing_dates = missing_by_date[missing_by_date >= 0.3]

# 갯수 출력
print("결측치 비율 ≥ 0.3인 날짜 수:", len(high_missing_dates))

# 날짜와 비율 출력
print("\n결측치 비율 ≥ 0.3인 날짜 리스트:")
print(high_missing_dates.sort_values(ascending=False))

In [ ]:
# 4. 수익률 기초 통계

# 전체 수익률 데이터를 1차원 + 결측치 제거
returns = df.values.flatten()
returns = returns[~pd.isna(returns)]

# 기초 통계량 계산
print("수익률 기초 통계:")
print("평균:", returns.mean())
print("중앙값:", pd.Series(returns).median())
print("표준편차:", returns.std())
print("최소값:", returns.min())
print("최대값:", returns.max())

In [ ]:
# 5. 히스토그램

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# ±30% 단순수익률 기준 로그수익률 범위 선택
returns_clean = returns[(returns > -0.357) & (returns < 0.262)]

# 히스토그램
plt.figure(figsize=(8,5))  # 그래프 크기 설정
sns.histplot(returns_clean, bins=100, kde=True, color='skyblue')  # 히스토그램
plt.title("Distribution of Log Returns") # 로그수익 분포
plt.xlabel("Log Rate of Return") # 로그수익률
plt.ylabel("Frequency") # 빈도
plt.show()

In [ ]:
# 6. 0 값 비율

# 전체 0 개수
zero_count = (df == 0).sum().sum()  # 각 셀 비교 후 합계

# 전체 데이터 수
total_count = df.size  # 행*열 전체 관측치

# 0 비율 계산
zero_ratio = zero_count / total_count

print("전체 0 값 개수:", zero_count)
print("전체 데이터 개수:", total_count)
print("0 값 비율:", zero_ratio*100,"%")

In [ ]:
# 7. 로그 수익률 하위 top10
df.stack().sort_values().head(10)

In [ ]:
# 로그 수익률 상위 top 10
df.stack().sort_values().tail(10)

In [ ]:
#EDA of kospi.csv

# 1. 기초 통계

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv("kospi.csv", index_col=0, parse_dates=True)  # 날짜 -> index

print("기초 통계")
print(df.describe())

In [ ]:
# 2. 코스피 로그 수익률 시계열 그래프
plt.figure(figsize=(12,4))
plt.plot(df.index, df['rm'], color='blue')  # 날짜별 수익률 라인
plt.title("KOSPI rate of retrun") #코스피 수익률
plt.xlabel("date") # 날짜
plt.ylabel("Log Rate of Return") # 로그 수익률
plt.show()

In [ ]:
# 3. 히스토그램
plt.figure(figsize=(6,4))
sns.histplot(df['rm'], bins=50, kde=True, color='green')  # 히스토그램 + KDE
plt.title("KOSPI rate of retrun") # 코스피 수익률
plt.xlabel("Log Rate of Return") # 로그 수익률
plt.show()

In [ ]:
# 4. 이동 표준편차

import numpy as np

# ±30% 확인
threshold = 0.3  # 로그수익률 기준 ±0.3
outliers = df[df['rm'].abs() > threshold]  # 절대값이 0.3 초과인 행
print("±30% 이상 수익률(극단값) 개수:", len(outliers))
print(outliers)

# 이동 표준편차 계산
window = 20  # 20일 이동표준편차
df['volatility'] = df['rm'].rolling(window=window).std()  # rolling std 계산

# 이동 변동성 시각화
plt.figure(figsize=(12,4))
plt.plot(df.index, df['volatility'], color='red')
plt.title("20-day movement volatility") # 20일 이동 변동성
plt.xlabel("date") # 날짜
plt.ylabel("volatility") # 변동성
plt.show()

In [ ]:
# 5. 연도별 변동성
df['year'] = df.index.year
df['month'] = df.index.month

# 연도별 변동성 평균
annual_vol = df.groupby('year')['rm'].std()
plt.figure(figsize=(8,4))
annual_vol.plot(kind='bar', color='orange')
plt.title("volatility by year") # 연도별 변동성 (표준편차)
plt.xlabel("year") # 연도
plt.ylabel("volatility") # 변동성
plt.show()

In [ ]:
# 월별 변동성 평균
monthly_vol = df.groupby('month')['rm'].std()
plt.figure(figsize=(8,4))
monthly_vol.plot(kind='bar', color='purple')
plt.title("monthly volatility)") # 월별 변동성 (표준편차)
plt.xlabel("month")
plt.ylabel("volatility")
plt.show()

In [ ]:
# 샤프비율 (로그수익률 기준)

import numpy as np

# 로그수익률
returns = df['rm']

# 평균 로그수익률
mean_return = returns.mean()

# 로그수익률 표준편차
std_return = returns.std()

sharpe_ratio = mean_return / std_return

print("샤프비율:", sharpe_ratio)

In [ ]:
# 최대 낙폭 (MDD)

# 누적 로그수익률
cum_log_return = returns.cumsum()

# 누적 수익률을 지수화 (실제 수익률 곡선)
cum_return = np.exp(cum_log_return)

# 과거 최고점 계산
running_max = cum_return.cummax()

# Drawdown 계산 (고점 대비 하락률)
drawdown = (cum_return - running_max) / running_max

mdd = drawdown.min()

print("MDD:", mdd)